# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [38]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Ignores warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [41]:
# Some additional minor data cleaning of city names for use later
city_data_df['City'] = city_data_df['City'].str.replace('-', ' ', regex=False)
city_data_df['City'] = city_data_df['City'].astype(str)

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,quatre cocos,-20.2078,57.7625,25.06,87,100,4.58,MU,1731267496
1,1,longyearbyen,78.2186,15.6401,-3.09,86,75,0.00,SJ,1731267280
2,2,udachny,66.4167,112.4000,-18.01,98,85,1.93,RU,1731267428
3,3,adamstown,-25.0660,-130.1015,22.06,81,100,6.18,PN,1731267411
4,4,dingwall,57.5953,-4.4272,6.99,91,2,3.39,GB,1731267501


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [42]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    size='Humidity',
    frame_width=500,
    frame_height = 350,
    scale=0.6,
    color='City',
    alpha=0.4
)

# Display the map
city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [77]:
# Narrow down cities that fit criteria and drop any results with null values
# Additional Comment: Here's the filters I decided on. I like colder temperatures so I looked for anything below 20 degrees celsius (~68 degrees
#                     fahrenheit) and above 10 degrees celsius (~50 degrees fahrenheit). The second filter layer looks for cities that report
#                     a humidity of less than 72%. I did one last layer of filtering for any cities that report wind speeds of less than 4 m/s.
city_Temp_Filter = city_data_df[city_data_df['Max Temp'].between(10,20)]
city_Cloud_Filter = city_Temp_Filter[city_Temp_Filter['Humidity'] < 72]
city_Wind_Filter = city_Cloud_Filter[city_Cloud_Filter['Wind Speed'] < 4]

# Drop any rows with null values
city_Wind_Filter.dropna(inplace=True)

# Display sample data
city_Wind_Filter

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,chatra,24.2167,84.8667,19.02,62,0,1.55,IN,1731267554
52,52,gembu,6.7000,11.2667,17.02,71,41,1.56,NG,1731267559
69,69,newman,37.3138,-121.0208,18.35,41,42,0.89,US,1731267580
75,75,north logan,41.7694,-111.8047,10.76,49,91,1.32,US,1731267587
88,88,albany,42.6001,-73.9662,12.23,41,100,1.34,US,1731267316
103,103,bethel,41.3712,-73.4140,14.39,45,0,3.60,US,1731267370
107,107,qa'en,33.7265,59.1844,10.87,51,2,0.96,IR,1731267625
118,118,al hasakah,36.5024,40.7477,18.30,31,96,2.60,SY,1731267563
149,149,fethiye,36.6217,29.1164,15.76,70,100,2.06,TR,1731267674
153,153,viedma,-40.8135,-62.9967,19.11,22,100,2.44,AR,1731267023


### Step 3: Create a new DataFrame called `hotel_df`.

In [78]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_Wind_Filter[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
48,chatra,IN,24.2167,84.8667,62,
52,gembu,NG,6.7000,11.2667,71,
69,newman,US,37.3138,-121.0208,41,
75,north logan,US,41.7694,-111.8047,49,
88,albany,US,42.6001,-73.9662,41,
103,bethel,US,41.3712,-73.4140,45,
107,qa'en,IR,33.7265,59.1844,51,
118,al hasakah,SY,36.5024,40.7477,31,
149,fethiye,TR,36.6217,29.1164,70,
153,viedma,AR,-40.8135,-62.9967,22,


In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,City,Country,Lat,Lng,Humidity,Hotel Name
43,saint-pierre,RE,-21.3393,55.4781,61,
66,badger,US,64.8000,-147.5333,37,
102,ust'-kulom,RU,61.6864,53.6902,43,
120,emerald,AU,-23.5333,148.1667,28,
130,newman,US,37.3138,-121.0208,51,
138,fort bragg,US,35.1390,-79.0060,80,
146,laguna,US,38.4210,-121.4238,52,
160,nova vicosa,BR,-17.8919,-39.3719,87,
170,fort myers shores,US,26.7092,-81.7459,95,
176,trinity beach,AU,-16.7888,145.6968,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
badger - nearest hotel: Clarion Hotel & Suites
ust'-kulom - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
newman - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
laguna - nearest hotel: Holiday Inn Express & Suites
nova vicosa - nearest hotel: Pousada Pontal da Barra
fort myers shores - nearest hotel: Comfort Inn Fort Myers
trinity beach - nearest hotel: Blue Lagoon Resort
racine - nearest hotel: No hotel found
six-fours-les-plages - nearest hotel: Kyriad Direct La Seyne-sur-Mer
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
posto fiscal rolim de moura - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
roswell - nearest hotel: Best Western Plus Hotel
hell-ville - nearest hotel: ABUD HOTEL
sona - nearest hotel: Al Bosco
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by W

,City,Country,Lat,Lng,Humidity,Hotel Name
43,saint-pierre,RE,-21.3393,55.4781,61,Tropic Hotel
66,badger,US,64.8000,-147.5333,37,Clarion Hotel & Suites
102,ust'-kulom,RU,61.6864,53.6902,43,No hotel found
120,emerald,AU,-23.5333,148.1667,28,Emerald Central Hotel
130,newman,US,37.3138,-121.0208,51,No hotel found
138,fort bragg,US,35.1390,-79.0060,80,Airborne Inn Lodging
146,laguna,US,38.4210,-121.4238,52,Holiday Inn Express & Suites
160,nova vicosa,BR,-17.8919,-39.3719,87,Pousada Pontal da Barra
170,fort myers shores,US,26.7092,-81.7459,95,Comfort Inn Fort Myers
176,trinity beach,AU,-16.7888,145.6968,44,Blue Lagoon Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)